In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import os
import sys

import matplotlib.pyplot as plt
from matplotlib import colors

from ez_data import *
from pyPulses.utils import sweepMeasureParallelepiped, SweepMeasureParallelepipedConfig

If you wish to use GPIB instruments, make sure NI488.2 is installed.


In [2]:
# Create the Dataset Object
path = r'data/demo'
DS = Dataset(path, storage_backend = 'hdf5')

variable_attrs = dict(
    Vtg = {'long_name': r'$V_{tg}$', 'units': r'V'},
    Vbg = {'long_name': r'$V_{bg}$', 'units': r'V'},
    R   = {'long_name': r'R', 'units': r'$\Omega$'}
)

global_attrs = dict(
    B = {'long_name': r'B', 'units': r'T', 'value': 0.0}
)

DS.add_variable_attrs(variable_attrs)
DS.add_global_attrs(global_attrs)

In [5]:
# Set up the sweep
swept_names = ("Vtg", "Vbg")
measured_names = ("R","timestamp")

def post_callback(index, coords, data):
    coord_dict = dict(zip(swept_names, coords))
    data_dict = dict(zip(measured_names, data))
    DS.add_point(index, coord_dict, data_dict)

origin = [5.0, 5.0]
endpoints = np.array([[6.0, 6.0],
                      [6.0,  4.0]])
shape = [101, 11]

def dummy_set(val): pass
def dummy_get(): return np.random.exponential(scale=1e6)

para_config = SweepMeasureParallelepipedConfig(
    setters=(dummy_set, dummy_set),
    getters=(dummy_get, datetime.datetime.now),
    swept_name=swept_names,
    measured_name=measured_names,
    time_per_point=0.01,
    origin=origin,
    endpoints=endpoints,
    shape=shape,
    post_callback=post_callback,
    retain_return=False
)

In [6]:
# Run the sweep
sweepMeasureParallelepiped(para_config)
# write metadata and flush any unsaved data
DS.write_metadata()
DS.flush()

In [3]:
# Load the data as a pandas dataframe

df = DS.load_dataframe()
df

,dim0,dim1,Vtg,Vbg,R,timestamp
0,0,0,5.0,5.0,2.513220e+05,2025-06-05 17:30:05.274288
1,0,1,5.1,4.9,2.886071e+06,2025-06-05 17:30:05.284809
2,0,2,5.2,4.8,4.050568e+04,2025-06-05 17:30:05.295327
3,0,3,5.3,4.7,2.402875e+05,2025-06-05 17:30:05.305857
4,0,4,5.4,4.6,2.079612e+05,2025-06-05 17:30:05.316129
...,...,...,...,...,...,...
106,100,6,6.6,5.4,2.283610e+06,2025-06-05 17:30:17.330040
107,100,7,6.7,5.3,2.290841e+06,2025-06-05 17:30:17.340615
108,100,8,6.8,5.2,4.201432e+03,2025-06-05 17:30:17.351645
109,100,9,6.9,5.1,9.773737e+05,2025-06-05 17:30:17.362237


In [4]:
# Load the data as an xarray Dataset

ds = DS.to_xarray()
ds

<xarray.Dataset> Size: 36kB
Dimensions:    (dim0: 101, dim1: 11)
Coordinates:
    Vtg        (dim0, dim1) float64 9kB 5.0 5.1 5.2 5.3 5.4 ... 6.7 6.8 6.9 7.0
    Vbg        (dim0, dim1) float64 9kB 5.0 4.9 4.8 4.7 4.6 ... 5.3 5.2 5.1 5.0
Dimensions without coordinates: dim0, dim1
Data variables:
    R          (dim0, dim1) float64 9kB 2.513e+05 2.886e+06 ... 3.027e+05
    timestamp  (dim0, dim1) datetime64[ns] 9kB 2025-06-05T17:30:05.274288 ......
Attributes:
    B:        {'long_name': 'B', 'units': 'T', 'value': 0.0}

In [9]:
# Plot the data in the xarray dataset as an example
%matplotlib qt5

ds["R"].plot.pcolormesh(x="Vtg", y ="Vbg", cmap = 'inferno', norm = colors.LogNorm(1e3, 1e6))
plt.show()

In [8]:
apply_linear_transform(df, np.array([[1, 1],[1, -1]]), ["Vtg", "Vbg"], ("n", "D"))

,dim0,dim1,Vtg,Vbg,R,timestamp,n,D
0,0,0,5.0,5.0,1.327032e+05,2025-06-05 17:26:40.339665,10.0,0.0
1,0,1,5.1,4.9,9.576991e+05,2025-06-05 17:26:40.350519,10.0,0.2
2,0,2,5.2,4.8,1.566948e+06,2025-06-05 17:26:40.361063,10.0,0.4
3,0,3,5.3,4.7,2.329612e+06,2025-06-05 17:26:40.371584,10.0,0.6
4,0,4,5.4,4.6,4.643900e+05,2025-06-05 17:26:40.382617,10.0,0.8
...,...,...,...,...,...,...,...,...
1106,100,6,6.6,5.4,1.530104e+06,2025-06-05 17:26:52.082261,12.0,1.2
1107,100,7,6.7,5.3,2.298090e+05,2025-06-05 17:26:52.092780,12.0,1.4
1108,100,8,6.8,5.2,5.948395e+05,2025-06-05 17:26:52.103300,12.0,1.6
1109,100,9,6.9,5.1,1.407759e+06,2025-06-05 17:26:52.113820,12.0,1.8


In [9]:
ds = apply_linear_transform(ds, np.array([[1, 1],[1, -1]]), ["Vtg", "Vbg"], ("n", "D"))
ds

<xarray.Dataset> Size: 53kB
Dimensions:    (dim0: 101, dim1: 11)
Coordinates:
    Vtg        (dim0, dim1) float64 9kB 5.0 5.1 5.2 5.3 5.4 ... 6.7 6.8 6.9 7.0
    Vbg        (dim0, dim1) float64 9kB 5.0 4.9 4.8 4.7 4.6 ... 5.3 5.2 5.1 5.0
    n          (dim0, dim1) float64 9kB 10.0 10.0 10.0 10.0 ... 12.0 12.0 12.0
    D          (dim0, dim1) float64 9kB 0.0 0.2 0.4 0.6 0.8 ... 1.4 1.6 1.8 2.0
Dimensions without coordinates: dim0, dim1
Data variables:
    R          (dim0, dim1) float64 9kB 1.327e+05 9.577e+05 ... 2.962e+06
    timestamp  (dim0, dim1) datetime64[ns] 9kB 2025-06-05T17:26:40.339665 ......
Attributes:
    B:        {'long_name': 'B', 'units': 'T', 'value': 0.0}

In [10]:
# Plot the data in the xarray dataset as an example
%matplotlib qt5

ds["R"].plot.pcolormesh(x="n", y ="D", cmap = 'inferno', norm = colors.LogNorm(1e3, 1e6))
plt.show()

In [11]:
def nD_from_V(Vtg, Vbg):
    return Vtg + Vbg, Vtg - Vbg

In [12]:
apply_transform(ds, nD_from_V, ["Vtg", "Vbg"], ("n", "D"))

<xarray.Dataset> Size: 53kB
Dimensions:    (dim0: 101, dim1: 11)
Coordinates:
    Vtg        (dim0, dim1) float64 9kB 5.0 5.1 5.2 5.3 5.4 ... 6.7 6.8 6.9 7.0
    Vbg        (dim0, dim1) float64 9kB 5.0 4.9 4.8 4.7 4.6 ... 5.3 5.2 5.1 5.0
    n          (dim0, dim1) float64 9kB 10.0 10.0 10.0 10.0 ... 12.0 12.0 12.0
    D          (dim0, dim1) float64 9kB 0.0 0.2 0.4 0.6 0.8 ... 1.4 1.6 1.8 2.0
Dimensions without coordinates: dim0, dim1
Data variables:
    R          (dim0, dim1) float64 9kB 1.327e+05 9.577e+05 ... 2.962e+06
    timestamp  (dim0, dim1) datetime64[ns] 9kB 2025-06-05T17:26:40.339665 ......
Attributes:
    B:        {'long_name': 'B', 'units': 'T', 'value': 0.0}

In [13]:
apply_transform(df, nD_from_V, ["Vtg", "Vbg"], ("n", "D"))

,dim0,dim1,Vtg,Vbg,R,timestamp,n,D
0,0,0,5.0,5.0,1.327032e+05,2025-06-05 17:26:40.339665,10.0,0.0
1,0,1,5.1,4.9,9.576991e+05,2025-06-05 17:26:40.350519,10.0,0.2
2,0,2,5.2,4.8,1.566948e+06,2025-06-05 17:26:40.361063,10.0,0.4
3,0,3,5.3,4.7,2.329612e+06,2025-06-05 17:26:40.371584,10.0,0.6
4,0,4,5.4,4.6,4.643900e+05,2025-06-05 17:26:40.382617,10.0,0.8
...,...,...,...,...,...,...,...,...
1106,100,6,6.6,5.4,1.530104e+06,2025-06-05 17:26:52.082261,12.0,1.2
1107,100,7,6.7,5.3,2.298090e+05,2025-06-05 17:26:52.092780,12.0,1.4
1108,100,8,6.8,5.2,5.948395e+05,2025-06-05 17:26:52.103300,12.0,1.6
1109,100,9,6.9,5.1,1.407759e+06,2025-06-05 17:26:52.113820,12.0,1.8


In [ ]:
# NOT RECOMMENDED

import mpld3

%matplotlib qt5

fig, ax = plt.subplots()
ds["R"].plot.pcolormesh(ax = ax, x="n", y ="D", cmap = 'inferno', norm = colors.LogNorm(1e3, 1e6))

html_str = mpld3.fig_to_html(fig)
with open("interactive_plot.html", "w") as Html_file:
    Html_file.write(html_str)

plt.show()

# Colorbar interactivity is broken. Prefer just saving static images.

c:\Users\avdif\Documents\ez_data\.venv\Lib\site-packages\mpld3\mplexporter\utils.py:300: MatplotlibDeprecationWarning: The converter attribute was deprecated in Matplotlib 3.10 and will be removed in 3.12. Use get_converter and set_converter methods instead.
  isinstance(axis.converter, matplotlib.dates._SwitchableDateConverter)
c:\Users\avdif\Documents\ez_data\.venv\Lib\site-packages\mpld3\mplexporter\utils.py:303: MatplotlibDeprecationWarning: The converter attribute was deprecated in Matplotlib 3.10 and will be removed in 3.12. Use get_converter and set_converter methods instead.
  isinstance(axis.converter, matplotlib.dates.DateConverter)
c:\Users\avdif\Documents\ez_data\.venv\Lib\site-packages\mpld3\mplexporter\utils.py:306: MatplotlibDeprecationWarning: The converter attribute was deprecated in Matplotlib 3.10 and will be removed in 3.12. Use get_converter and set_converter methods instead.
  isinstance(axis.converter, matplotlib.dates.ConciseDateConverter)
c:\Users\avdif\Documen

In [2]:
# Smarty Sweep / Special Measure Compatibility

sm_data_folder = r'C:\Users\avdif\MIT Dropbox\Andrew DiFabbio\Capacitance_2059\S91_2059He3\data'

def Vraw_to_V(Vtg, Vc):
    return Vtg - Vc, -Vc

q = 1.60217663e-19      # C
ε0 = 8.8541878188e-12   # F⋅m−1
Ct = 7.4397375e+15 * q  # F⋅m−2
Cb = 7.6688751e+15 * q  # F⋅m−2
n1 = 7.7093901e+15      # m−2
n0 = 2.9623501e+15      # m−2
D0 = -6730550           # V⋅m−1
def V_to_nD(Vtg, Vbg):
    n = (Ct*Vtg + Cb*Vbg)/q - n0
    D = (Ct*Vtg - Cb*Vbg)/(2*ε0) - D0
    return n, D

# This iterator pattern for the indices is common of smarty sweep /
# special measure data (often we split files by lines)
dim0 = 0
def idx_func(df):
    # Convert from raw voltages to effective Vtg and Vbg
    df = apply_transform(df, Vraw_to_V, ('Vtg', 'Vc'), ('Vtg', 'Vbg'))
    # Add n and D columns
    df = apply_transform(df, V_to_nD, ('Vtg', 'Vbg'), ('n', 'D'))
    # Add index
    global dim0
    idx0 = dim0
    dim0 += 1
    df = df.reset_index(names = 'dim1')
    df['dim0'] = idx0
    return df

col_names = {
    'timestamp' : '#Timestamp',
    'Xoff'      : 'X_{offbal} (V)',
    'Yoff'      : 'Y_{offbal} (V)',
    'Cex'       : 'C/C_{std}',
    'Closs'     : 'C_{loss}/C_{std}',
    'Vc'        : 'V_{Contacts} (V)',
    'Ic'        : 'I_{Contacts} (A)',
    'Vtg'       : 'V_{TG} (V)',
    'Itg'       : 'I_{TG} (A)',
    'f'         : '*Freq (Hz)',
    'B'         : '*B (T)',
    'Vexc'      : '*Excitation (V)'
}

sm_df = dat_to_pandas(sm_data_folder, "_scan149_map_340mK.dat", idx_func, col_names)
sm_df

,dim1,timestamp,Xoff,Yoff,Cex,Closs,Vc,Ic,Vtg,Itg,f,B,Vexc,Vbg,n,D,dim0
0,0,2025-06-05T08:19:41.970,-0.000026,-0.000025,1.01055,0.046859,5.70916,0,6.56504,0,7777780.0,0,0.015,-5.70916,2.096989e+15,8.447613e+08,0
1,1,2025-06-05T08:19:42.404,-0.000025,-0.000023,1.01054,0.045868,5.70886,0,6.56534,0,7777780.0,0,0.015,-5.70886,2.101522e+15,8.447606e+08,0
2,2,2025-06-05T08:19:42.828,NaN,NaN,NaN,NaN,5.70856,0,6.56564,0,7777780.0,0,0.015,-5.70856,2.106054e+15,8.447600e+08,0
3,3,2025-06-05T08:19:43.251,0.001753,-0.001143,2.01085,0.040909,5.70826,0,6.56594,0,7777780.0,0,0.015,-5.70826,2.110587e+15,8.447594e+08,0
4,4,2025-06-05T08:19:43.660,0.001753,-0.001143,2.01058,0.040734,5.70795,0,6.56625,0,7777780.0,0,0.015,-5.70795,2.115271e+15,8.447587e+08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,996,2025-06-05T15:45:05.503,-0.000021,-0.000021,1.01152,0.044302,5.23985,0,6.69875,0,7777780.0,0,0.015,-5.23985,6.690836e+15,8.211986e+08,60
61057,997,2025-06-05T15:45:05.912,-0.000022,-0.000020,1.01125,0.044027,5.23955,0,6.69905,0,7777780.0,0,0.015,-5.23955,6.695369e+15,8.211979e+08,60
61058,998,2025-06-05T15:45:06.323,-0.000022,-0.000022,1.01159,0.044708,5.23925,0,6.69935,0,7777780.0,0,0.015,-5.23925,6.699901e+15,8.211973e+08,60
61059,999,2025-06-05T15:45:06.725,-0.000021,-0.000023,1.01212,0.044856,5.23895,0,6.69965,0,7777780.0,0,0.015,-5.23895,6.704434e+15,8.211967e+08,60


In [3]:
coord_attr = {
    'Vtg'   : {'long_name': r'$V_{tg}$', 'units': 'V'},
    'Vbg'   : {'long_name': r'$V_{bg}$', 'units': 'V'},
    'n'     : {'long_name': r'$n$', 'units': r'm$^{-2}$'},
    'D'     : {'long_name': r'$D$', 'units': 'V/m'},
    'B'     : {'long_name': r'$B$', 'units': 'T'}
}
data_attr = {
    'Xoff'  : {'long_name': r"X_{off}", 'units': 'V'},
    'Yoff'  : {'long_name': r"Y_{off}", 'units': 'V'},
    'Cex'   : {'long_name': r"$C_{ex}/C_{std}$"},
    'Closs' : {'long_name': r"$C_{loss}/C_{std}$"},
    'timestamp': {'long_name': 'time'}
}
global_attr = {
    'f'     : {'long_name': r'$f$', 'units': 'Hz', 'value': sm_df['f'].mean()},
    'Vexc'  : {'long_name': r'$V_{exc}$', 'units': 'V', 'value': sm_df['Vexc'].mean()}
}
# Unused for coord_attr and data_attr, but also not indices
ignored_attr = ['Ic', 'Itg', 'f', 'Vexc', 'Vc']

sm_ds = pandas_to_xarray(sm_df, coord_attr, data_attr, global_attr, ignored_attr)
sm_ds

<xarray.Dataset> Size: 5MB
Dimensions:    (dim1: 1001, dim0: 61)
Coordinates:
    Vtg        (dim1, dim0) float64 488kB 6.565 6.562 6.559 ... 6.706 6.703 6.7
    Vbg        (dim1, dim0) float64 488kB -5.709 -5.706 -5.704 ... -5.241 -5.239
    n          (dim1, dim0) float64 488kB 2.097e+15 2.097e+15 ... 6.709e+15
    D          (dim1, dim0) float64 488kB 8.448e+08 8.444e+08 ... 8.212e+08
    B          (dim1, dim0) float64 488kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Dimensions without coordinates: dim1, dim0
Data variables:
    Xoff       (dim1, dim0) float64 488kB -2.6e-05 1.433e-05 ... -2.105e-05
    Yoff       (dim1, dim0) float64 488kB -2.455e-05 2.21e-05 ... -2.189e-05
    Cex        (dim1, dim0) float64 488kB 1.011 1.015 1.015 ... 1.011 1.012
    Closs      (dim1, dim0) float64 488kB 0.04686 0.0178 ... 0.04437 0.04452
    timestamp  (dim1, dim0) datetime64[ns] 488kB 2025-06-05T08:19:41.970000 ....
Attributes:
    f:        {'long_name': '$f$', 'units': 'Hz', 'value': np.float64(7777780...
    Vexc:     {'long_name': '$V_{exc}$', 'units': 'V', 'value': np.float64(0....

In [6]:
%matplotlib qt5
sm_ds['Cex'].plot.pcolormesh(x='n', y='D', vmin = 1.0, vmax = 1.05, cmap = 'inferno')
plt.show()

In [5]:
%matplotlib qt5
sm_ds['Cex'].plot.pcolormesh(x='Vtg', y='Vbg', vmin = 1.0, vmax = 1.05, cmap = 'inferno')
plt.show()

In [69]:
sm_ds.where((sm_ds['D'] > 8.395e8) & (sm_ds['D'] < 8.4e8), drop = True)['Cex'].plot.line(x = 'n', color = 'r')
plt.show()

In [70]:
sm_ds.where((sm_ds['Vtg'] > 6.52) & (sm_ds['Vtg'] < 6.524), drop = True)['Cex'].plot.line(x = 'Vbg', color = 'r')
plt.show()